In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, RobertaTokenizer, DistilBertTokenizer
from transformers import BertForSequenceClassification, RobertaForSequenceClassification, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
import numpy as np

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [19]:

# # Preprocess messages
# nltk.download('punkt')
# nltk.download('stopwords')

# Load dataset
data = pd.read_csv('ScamDataset.csv', delimiter=',', names=['message', 'Label'])


# Displaying the shape of the dataset before removing duplicates
original_shape = data.shape

# Removing duplicate rows
data = data.drop_duplicates()

# Shape after removing duplicates
new_shape = data.shape

original_shape, new_shape

# stemmer = PorterStemmer()
# stop_words = set(stopwords.words('english'))

# def preprocess(text):
#     tokens = word_tokenize(text)
#     stemmed = [stemmer.stem(word) for word in tokens if word.isalpha() and word not in stop_words]
#     return ' '.join(stemmed)

# data['processed_message'] = data['message'].apply(preprocess)
# data['processed_message']


data['Label'] = data['Label'].map({'normal': 0, 'fraud': 1})

texts = data['message'].tolist()
labels = data['Label'].tolist()

# Split the dataset
train_texts, temp_texts, train_labels, temp_labels = train_test_split(texts, labels, test_size=0.4,random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5,random_state=42)

print (train_texts)

["Same here,  but I consider walls and bunkers and shit important just because I never play on peaceful but I guess your place is high enough that it don't matter", 'Important notice: Your tax refund requires urgent attention. Act without delay to avoid issues. Ref: 7066', 'I am taking half day leave bec i am not well', 'Or ill be a little closer like at the bus stop on the same street', 'species and fraudulent activities carried out using your identity. The court has issued a suspension notice along with a warrant against your name. Ignoring this message will be an intentional second attempt. To avoid initial appearance before a magistrate, judge, or grand jury for federal criminal offense, this is the final attempt to reach you. For more information, press 1 and hold for the next developmental officer. ', 'Looking forward to your party! Should I bring anything?', 'You are awarded a SiPix Digital Camera! call 09061221061 from landline. Delivery within 28days. T Cs Box177. M221BP. 2yr 

In [20]:
class ScamDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=128)
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(int(self.labels[idx]))
        return item

    def __len__(self):
        return len(self.labels)


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = (predictions == labels).mean()
    
    return {"accuracy": accuracy}

# Get predictions and plot confusion matrix
def get_predictions_and_plot_confusion_matrix(trainer, test_dataset, model_name):
    predictions = trainer.predict(test_dataset)
    preds = np.argmax(predictions.predictions, axis=1)
    true_labels = test_dataset.data['Label'].values

    # Plot confusion matrix
    labels = ['normal', 'fraud']
    cm = confusion_matrix(true_labels, preds, labels=[0, 1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f'Confusion Matrix: {model_name}')
    plt.show()

# Define function for training and evaluation
def train_and_evaluate(model_name):
    print(f"Training and evaluating {model_name}...")
    if model_name == 'bert':
        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
    elif model_name == 'roberta':
        tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
        model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)
    elif model_name == 'distilbert':
        tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
        model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
   

    else:
        raise Exception("Model not supported")

    train_dataset = ScamDataset(train_texts, train_labels, tokenizer)
    val_dataset = ScamDataset(val_texts, val_labels, tokenizer)
    test_dataset = ScamDataset(test_texts, val_labels, tokenizer)


    training_args = TrainingArguments(
        output_dir='./results',          
        learning_rate = 1e-4,
        per_device_train_batch_size = 8,
        per_device_eval_batch_size = 8,
        num_train_epochs = 2,
        weight_decay = 0.01,
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        load_best_model_at_end = True 
    )

    trainer = Trainer(
        model=model,                         
        args=training_args,                  
        train_dataset=train_dataset,         
        eval_dataset=val_dataset,  
        compute_metrics=compute_metrics,
    )

    trainer.train()
    
    get_predictions_and_plot_confusion_matrix(trainer, test_dataset, model_name)

    return trainer.evaluate()





In [21]:
model_accuracies = {}
model_name = 'bert'

eval_results = train_and_evaluate(model_name)
model_accuracies[model_name] = eval_results['eval_accuracy']


Training and evaluating bert...


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  5%|▍         | 110/2286 [00:51<16:25,  2.21it/s]

KeyboardInterrupt: 

In [ ]:
model_name = 'roberta'

eval_results = train_and_evaluate(model_name)
model_accuracies[model_name] = eval_results['eval_accuracy']


In [ ]:
model_name = 'distilbert'

eval_results = train_and_evaluate(model_name)
model_accuracies[model_name] = eval_results['eval_accuracy']

In [ ]:
print('Model Accuracies:')
for model_name, accuracy in model_accuracies.items():
    print(f"{model_name}: {accuracy}")


In [ ]:
import matplotlib.pyplot as plt
# Plotting the accuracies using a more distinct graph style
plt.figure(figsize=(10, 6))

# Create a bar graph with distinct colors and edge color
bars = plt.bar(model_accuracies.keys(), model_accuracies.values(), color=['blue', 'green', 'red'], edgecolor='black')

# Add data labels above each bar for clarity
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval + 0.001, round(yval, 3), ha='center', va='bottom')

plt.xlabel('Model', fontsize=14)
plt.ylabel('Accuracy', fontsize=14)
plt.title('Comparison of Model Accuracies on scam Detection', fontsize=16)
plt.ylim(0.99, 1)  # Set y-axis limit to make differences more distinct
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()